In [1]:
import os
import numpy as np
import scipy
from plotly import express as px

from eegsc.utils.io import read_raw
from eegsc.utils.path import get_data_path
from eegsc.utils.experiments import create_sequence_dataset
from eegsc.preprocessing.filters import BandPassFilter, FFTFilter

In [2]:
data = read_raw(os.path.join(get_data_path(), 'raw', '1st_Day.mat'),
                data_types=['left_real', 'right_real'])

In [3]:
start_time = 5
signal_duration = 10 - start_time
bandpass_filter = BandPassFilter(signal_duration=signal_duration)

In [4]:
statistics, labels, person_idxs, times = create_sequence_dataset(
    data=data,
    bandpass_filter=bandpass_filter,
    start_time=start_time,
    is_len_equal=True,
    return_time=True
)
statistics.shape, labels.shape

((569, 32, 5094), (569,))

In [5]:
statistics *= 1e6

In [6]:
sig = statistics[0, 0]
time = times[0]

In [7]:
px.line(x=time, y=sig)

In [8]:
sample_rate = len(time) / (time[-1] - time[0])
fft = scipy.fft.rfft(sig)
freqs = scipy.fft.rfftfreq(len(time), 1 / sample_rate)

In [9]:
px.bar(x=freqs, y=np.abs(fft))

In [31]:
filtered_fft = fft.copy()
# filtered_fft[np.where(fft < 3000)] = 0
filtered_fft[np.where(np.abs(fft) < 3000)] = 0
filtered_sig = scipy.fft.irfft(filtered_fft)
px.line(x=time, y=filtered_sig)

In [11]:
sample_rate = len(time) / (time[-1] - time[0])
fft = scipy.fft.rfft(sig)
freqs = scipy.fft.rfftfreq(len(time), 1 / sample_rate)
px.bar(x=freqs, y=np.abs(fft) / np.abs(fft).max())

In [28]:
filtered_fft = fft.copy()
filtered_fft[np.where((np.abs(fft) / np.abs(fft).max()) < .5)] = 0
filtered_sig = scipy.fft.irfft(filtered_fft)
px.line(x=time, y=filtered_sig)

In [13]:
np.isnan(filtered_sig).any()

False

In [17]:
fft_filter = FFTFilter(.75)
filtered_sig = fft_filter.filter(np.array([[sig]]))[0, 0]
px.line(x=time, y=filtered_sig)

signal: [  0.19235745   0.91741876   2.2145077  ... -19.48436385 -17.53711505
 -14.91174014]
filtered_signal: [-2.27348268 -1.62084162 -0.99394367 ... -4.21313418 -3.5867497
 -2.93456759]
